In [ ]:
import json
from typing import Dict, Any

# ==============================
# Step 1: 自动生成 Prompt Schema
# ==============================
def generate_prompt_schema(requirement: str) -> Dict:
    """
    模拟 LLM 输出的 JSON Schema
    """
    if "天气" in requirement:
        schema = {
            "$schema": "https://json-schema.org/draft/2020-12/schema",
            "title": "WeatherQuerySchema",
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "用户要查询天气的城市，必填"},
                "date": {"type": "string", "description": "用户要查询天气的日期，允许相对日期"},
                "unit": {"type": "string", "enum": ["C", "F"], "description": "温度单位"}
            },
            "required": ["city"],
            "additionalProperties": False
        }
    else:
        schema = {"error": "暂不支持该需求"}
    return schema


requirement = "帮我设计一个查天气的对话系统，用户可以指定城市、日期、温度单位（摄氏度或华氏度），并且城市是必填字段。"
schema = generate_prompt_schema(requirement)

print("✅ 自动生成的 Prompt Schema：\n")
print(json.dumps(schema, indent=2, ensure_ascii=False))

# ===================================
# Step 2: Schema 驱动的多轮对话收集
# ===================================

# 初始化上下文
context_state = {k: None for k in schema["properties"].keys()}
context_state["unit"] = "C"  # 给一个默认值

def merge_slots(context: Dict[str, Any], overrides: Dict[str, Any]) -> Dict[str, Any]:
    merged = context.copy()
    for k, v in overrides.items():
        merged[k] = v
    return merged

def dialogue_step(user_input: str, context: Dict[str, Any]) -> Dict[str, Any]:
    overrides = {}
    if "北京" in user_input:
        overrides["city"] = "北京"
    if "上海" in user_input:
        overrides["city"] = "上海"
    if "华氏" in user_input:
        overrides["unit"] = "F"
    if "摄氏" in user_input:
        overrides["unit"] = "C"
    return merge_slots(context, overrides)

def generate_response(context: Dict[str, Any]) -> str:
    if not context["city"]:
        return "你想查询哪个城市的天气？"
    return f"{context['city']} 明天的温度是 22°{context['unit']}。"

# 非交互式模拟
test_inputs = ["帮我查北京天气", "改成华氏度", "查上海的"]
context = context_state.copy()
for i, user_input in enumerate(test_inputs, 1):
    context = dialogue_step(user_input, context)
    response = generate_response(context)
    print(f"[第{i}轮] 👤: {user_input}\n       🤖: {response}\n")

# ===================================
# Step 3: 拼装 Prompt（示意）
# ===================================
prompt_template = """
请根据以下槽位信息回答用户问题：
城市: {city}
日期: {date}
单位: {unit}
"""

final_prompt = prompt_template.format(**context)
print("📝 拼装后的 Prompt：")
print(final_prompt)
